In [1]:
import os
import pandas as pd
import re
import numpy as np

import collections
from operator import itemgetter
import pprint as pp
import re
from nltk.corpus import stopwords
import nltk.data
import time
import proj_base
#data from http://times.cs.uiuc.edu/~wang296/Data/
files = os.listdir('./Review_Texts')
#print(os.listdir('./Review_Texts'))
print(len(files))

1759


In [63]:
data = proj_base.getStandardData(numFiles=1)
data.shape

(124, 13)

In [3]:
data.head()

,Rooms,Date,Location,Service,Business service,Author,Check in / front desk,No. Helpful,Cleanliness,Content,Value,No. Reader,Overall
0,5.0,"Dec 23, 2008",5.0,5.0,NaN,selizabethm,5.0,NaN,5.0,wonderful time- even with the snow! what a gre...,4.0,NaN,5.0
1,4.0,"Nov 13, 2008",5.0,5.0,NaN,IndieLady,5.0,NaN,4.0,"lovely hotel, unique decor, friendly front des...",5.0,NaN,4.0
2,4.0,"Nov 11, 2008",3.0,NaN,4,Hilobb,5.0,NaN,4.0,"nice hotel, expensive parking we got a good de...",4.0,NaN,4.0
3,5.0,"Nov 4, 2008",5.0,5.0,5,Chianti_girl24,5.0,NaN,5.0,fabulous hotel location and service are great....,5.0,NaN,5.0
4,NaN,"Oct 18, 2008",NaN,NaN,NaN,hothearted,NaN,2,NaN,"loved the monaco! staff was amazing, with a sm...",NaN,2,5.0


In [ ]:
#mean of int columns
data.mean(axis=0)

In [ ]:
data["Content"][1]

In [4]:
#creating the vocab of all the words

def lintWord(w):
    regex = re.compile(r'(,|\(|\)|!|:|$|\.)')
    w = re.sub(regex, '', w)
    return w


allWords = ""
for r in data["Content"]:
    #add word to big content string
    allWords += r + " "
    
#split the string at spaces, keep only unique
words = set(allWords.split(" "))


vocab = list(set([lintWord(w) for w in words if not w in stopwords.words("english")]))

#n eed to remove stopwords again because some of them may have had punctuation 
# at the end and didnt get caught the first time
vocab = [w for w in vocab if not w in stopwords.words("english") and len(w) > 2]

#vocab


In [5]:
print(len(vocab))
print(len(set(vocab)))
print("i" in vocab)


3039
3039
False


In [ ]:

"""Algorithm: Aspect Segmentation Algorithm
Input: A collection of reviews {푑1, 푑2, . . . , 푑∣퐷∣)}, set of
aspect keywords {푇1, 푇2, . . . , 푇푘}, vocabulary V, selection
threshold p and iteration step limit I.
Output: Reviews split into sentences with aspect assignments.
Step 0: Split all reviews into sentences, 푋 =
{푥1, 푥2, . . . , 푥푀};
Step 1: Match the aspect keywords in each sentence
of X and record the matching hits for each aspect i in
퐶표푢푛푡(푖);
Step 2: Assign the sentence an aspect label by 푎푖 =
푎푟푔푚푎푥푖 퐶표푢푛푡(푖). If there is a tie, assign the sentence
with multiple aspects.
Step 3: Calculate chi^2 measure of each word (in V);
Step 4: Rank the words under each aspect with respect
to their chi^2value and join the top p words for each aspect
into their corresponding aspect keyword list 푇푖;
Step 5: If the aspect keyword list is unchanged or iteration
exceeds I, go to Step 6, else go to Step 1;
Step 6: Output the annotated sentences with aspect
assignments."""

In [6]:
#tokenizer to split sentences
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')


In [7]:
# initial seeds from LARA paper
seeds = {"Value" : ["value", "price", "quality","worth"],
         "Rooms" : ["room", "suite", "view", "bed"],
         "Location" : ["location", "traffic", "minute", "restaurant"],
         "Cleanliness" : ["clean", "dirty", "maintain", "smell"],
         "Check in / front desk": ["stuff", "check", "help", "reservation"],
         "Service" : ["service", "food", "breakfast", "buffet"],
         "Business service" : ["business", "center", "computer", "internet"]
        }


def aspectSegmentation(reviews, aspects, vocab=[], threshold=0, iterationLimit=3):
    #when we have the top chi-squared rated keywords, how many do we take
    keywordsToTake = 3
    
    #bootstrap iterations
    for i in range(0, iterationLimit):
        
        #print our current aspects
        print("begin bootstrapping iteration, aspect keywords: ")
        pp.pprint(aspects)
        print("\n\n\n")
        labeledSentences = []
        for r in reviews:
            #use the pickle tokenizer to split sentences
            sentences = tokenizer.tokenize(r)
            
            for s in sentences:
                
                theseAspects = collections.defaultdict(int)
                maxAspect = (0, "None")
                
                #for each aspect count how many times one of those aspect words appears
                for a in aspects:
                    for word in aspects[a]:
                        if " "+word+" " in s:
                            theseAspects[a] += 1
                
                #find the max occuring aspect for each sentence, take multiple if ties
                for a in theseAspects:
                    if theseAspects[a] > maxAspect[0]:
                        maxAspect = (theseAspects[a], a)
                    if theseAspects[a] == maxAspect[0] and a not in maxAspect:
                        #label it with multiple aspects
                        maxAspect = maxAspect + (a, )
                
                #add the sentence with labels
                labeledSentences.append((s, maxAspect[1:]))
            
        
        chiSquaredForAspects = collections.defaultdict(list)
        #calculate chi squared measure for each word in vocab
        """c1 is the number of times w occurs in sentences belonging
            to aspect a_i, c2 is the number of times w occurs
            in sentences not belonging to a_i, c3 is the number of sentences
            of aspect a_i that do not contain w, c4 is the number
            of sentences that neither belong to aspect a_i, nor contain
            word w, and C is the total number of word occurrences"""
        for w in vocab:
            for a in aspects:
                c = 0
                c_1 = 0
                c_2 = 0 
                c_3 = 0
                c_4 = 0

                for s in labeledSentences:
                    sentenceText = s[0]
                    sentenceAspects = s[1]

                    if " "+w+" " in sentenceText and a in sentenceAspects:
                        c_1 += 1
                    elif " "+w+" " in sentenceText and a not in sentenceAspects:
                        c_2 += 1
                    elif a in sentenceAspects and w not in sentenceText:
                        c_3 += 1
                    else:
                        c_4 += 1

                numer = ((1.0 * c_1 * c_4 - 1.0 * c_2 * c_3)**2)
                denom = (1.0*(c_1 + c_3) * 1.0 * (c_2 + c_4) * 1.0 * (c_1 + c_2) * 1.0 * (c_3 + c_4))
                #unreasonable use of 1.0's here to be safe
                if denom != 0:
                    csq = numer / denom
                    chiSquaredForAspects[a].append((w, csq))


        #have the chi squared aspects for each word in vocab, add top kewordsToTake for each aspect
        for a in chiSquaredForAspects:
            #make sure were not taking words we already have
            noDupes = [tup for tup in chiSquaredForAspects[a] if tup[0] not in aspects[a]]
            chiSquaredForAspects[a] = sorted(noDupes, key=itemgetter(1), reverse=True)[0:keywordsToTake]
            for t in chiSquaredForAspects[a]:
                if t[0] not in aspects[a] and t[0] != '':
                    aspects[a].append(t[0])
            
            
        
            #split into sentences
        
        #loop through again
    return labeledSentences
    #return labeledSentences

start = time.time()
sentencesWLabels = aspectSegmentation(data["Content"], seeds, vocab)
end = time.time()

print("done, time taken:", end-start)

begin bootstrapping iteration, aspect keywords: 
{'Business service': ['business', 'center', 'computer', 'internet'],
 'Check in / front desk': ['stuff', 'check', 'help', 'reservation'],
 'Cleanliness': ['clean', 'dirty', 'maintain', 'smell'],
 'Location': ['location', 'traffic', 'minute', 'restaurant'],
 'Rooms': ['room', 'suite', 'view', 'bed'],
 'Service': ['service', 'food', 'breakfast', 'buffet'],
 'Value': ['value', 'price', 'quality', 'worth']}




begin bootstrapping iteration, aspect keywords: 
{'Business service': ['business',
                      'center',
                      'computer',
                      'internet',
                      'access',
                      'wireless',
                      'work'],
 'Check in / front desk': ['stuff',
                           'check',
                           'help',
                           'reservation',
                           'real',
                           'set',
                           'efficient'],
 

In [ ]:
numToShow = 20
count = 0
for s in sentencesWLabels:
    if 'None' not in s[1]:
        print(s)
        print("\n")
        count += 1
    if count > numToShow:
        break

In [ ]:
#seeds will have changed, whats differet
seeds

In [ ]:
def seedInReview(review, seeds):
    for s in seeds:
        #print(s)
        if s in review:
            return True
    return False


reviewsWithSeeds = {}
for s in seeds:
    reviewsWithSeeds[s] = sum(data["Content"].apply(seedInReview, args = [seeds[s]]))
reviewsWithSeeds

In [66]:
def getContentLen(rev):
    return len(rev["Content"])

data.apply(getContentLen, axis=1)

0       422
1      1435
2       969
3       382
4       624
5      2890
6      2419
7      3550
8       250
9        66
10      418
11      641
12      791
13     1372
14       80
15      882
16      881
17      395
18      697
19      968
20      353
21     1283
22      295
23      508
24       79
25     1058
26      757
27     2432
28      293
29     2160
       ... 
94     1260
95     1251
96      617
97     1314
98      922
99      571
100     195
101     501
102     303
103    2236
104    1849
105     511
106     300
107    1226
108     575
109     354
110    1834
111    1569
112     707
113     238
114    1106
115     502
116    1715
117     658
118     921
119     449
120    3375
121    1479
122     662
123     642
dtype: int64

In [71]:
aspect = "Value"

def filterReviewSentencesByWords(rev, words):
    #tokenize review into sentences
    sentences = tokenizer.tokenize(rev["Content"])
    
    
    newContent = ""
    for s in sentences:
        wordlist = re.sub("[^a-zA-Z]"," ", s).split()
        intersect = set(wordlist).intersection(words)
        
        if len(intersect) != 0:
            newContent += s 

    
    if len(newContent) == 0:
        #so no sentences contain the rating? what do we do ehre
        pass
    rev["Content"] = newContent

    return rev

def filterReviewsByAspectWords(data, seeds):
    #we have our seeds, now let's only keep the sentences which are relevant to that aspect in each review
    aspectWords = set(seeds[aspect])
    
    
    data = data.apply(filterReviewSentencesByWords, axis=1, args=(aspectWords,))
    return data
    
data = filterReviewsByAspectWords(data[0:9], seeds)

{'reasonable', 'worth', 'value', '$217', 'per', '800', 'sign', 'quality', 'full', 'price', 'pricey', 'option', 'accessories'}
0 422
PST 0
0 422
PST 0
0 1435
PST 0
0 969
PST 0
0 382
PST 0
0 624
PST 0
0 2890
PST 0
490 2419
PST 490
639 3550
PST 639
0 250
PST 0


In [72]:
data.head(10)

,Rooms,Date,Location,Service,Business service,Author,Check in / front desk,No. Helpful,Cleanliness,Content,Value,No. Reader,Overall
0,5.0,"Dec 23, 2008",5.0,5.0,NaN,selizabethm,5.0,NaN,5.0,,4.0,NaN,5.0
1,4.0,"Nov 13, 2008",5.0,5.0,NaN,IndieLady,5.0,NaN,4.0,,5.0,NaN,4.0
2,4.0,"Nov 11, 2008",3.0,NaN,4.0,Hilobb,5.0,NaN,4.0,,4.0,NaN,4.0
3,5.0,"Nov 4, 2008",5.0,5.0,5.0,Chianti_girl24,5.0,NaN,5.0,,5.0,NaN,5.0
4,NaN,"Oct 18, 2008",NaN,NaN,NaN,hothearted,NaN,2.0,NaN,,NaN,2.0,5.0
5,3.0,"Oct 13, 2008",3.0,2.0,3.0,MauiDiver,2.0,1.0,5.0,,2.0,2.0,2.0
6,4.0,"Sep 15, 2008",5.0,3.0,5.0,Tulane86,1.0,1.0,5.0,"in comparison to the w, i spent about $45 more...",2.0,3.0,3.0
7,5.0,"Sep 15, 2008",5.0,5.0,NaN,kstenger,5.0,4.0,5.0,our daughter's 9th birthday was 09.13.08 and w...,5.0,4.0,5.0
8,5.0,"Sep 1, 2008",5.0,4.0,3.0,cantwaitNy,5.0,NaN,5.0,,5.0,NaN,5.0
